In [19]:
from datasets import load_dataset
from transformers import pipeline
import transformers as trf

import re  #preprocessing
import pandas as pd  #data handling
from time import time  #time-to-run
from collections import defaultdict  #word frequency

import spacy  #preprocessing/tokenization

import logging  #gensim logging
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

from tqdm import tqdm

import matplotlib.pyplot as plt

from sklearn.feature_extraction import text as sktext

import scipy as sp
import numpy as np

import sklearn as sk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import gensim
from gensim.models import Word2Vec, Doc2Vec
from joblib import dump, load

In [20]:
import torch
torch.cuda.is_available()

False

In [21]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", "3.0.0", split='train')
dataset = dataset.shard(num_shards=10, index=0)
dataset = dataset.remove_columns("id")
test_dataset = load_dataset("cnn_dailymail", "3.0.0", split='test')
test_dataset = test_dataset.shard(num_shards=10, index=0)
test_dataset = test_dataset.remove_columns("id")

WARNING - 02:12:20: Found cached dataset cnn_dailymail (C:/Users/mkrooted/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)
WARNING - 02:12:22: Found cached dataset cnn_dailymail (C:/Users/mkrooted/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)


In [22]:
from transformers import DataCollatorForSeq2Seq, AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
import evaluate

rouge = evaluate.load("rouge")

In [23]:
MODEL_PATH = "./t5-small/"

In [24]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file ./t5-small/config.json
Model config T5Config {
  "_name_or_path": "./t5-small/",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 

In [25]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=MODEL_PATH)

src: 
- https://huggingface.co/docs/transformers/tasks/summarization
- https://huggingface.co/docs/transformers/v4.27.2/en/main_classes/data_collator

In [26]:
prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=2048, truncation=True)

    labels = tokenizer(text_target=examples["highlights"], max_length=256, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [27]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

src:
- https://huggingface.co/docs/transformers/training#train-with-pytorch-trainer
- https://github.com/huggingface/notebooks/blob/main/examples/summarization.ipynb

In [28]:
import torch
torch.cuda.is_available()

False

In [29]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/28712 [00:00<?, ? examples/s]

In [30]:
test_tokenized_dataset = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1149 [00:00<?, ? examples/s]

In [31]:
tokenized_dataset

Dataset({
    features: ['article', 'highlights', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 28712
})

In [32]:


model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH)

loading configuration file ./t5-small/config.json
Model config T5Config {
  "_name_or_path": "./t5-small/",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping":

In [33]:
training_args = Seq2SeqTrainingArguments(
    output_dir="news_summarization",
    evaluation_strategy="epoch",      # "epoch": Evaluation is done at the end of each epoch.
    learning_rate=2e-5,               # The initial learning rate for AdamW optimizer
    per_device_train_batch_size=16,   # The batch size per GPU/TPU core/CPU for training
    per_device_eval_batch_size=16,    # The batch size per GPU/TPU core/CPU for evaluation
    weight_decay=0.01,     # The weight decay to apply (if not zero) to all layers except all bias and LayerNorm weights in AdamW optimizer.
                           # https://medium.com/analytics-vidhya/deep-learning-basics-weight-decay-3c68eb4344e9 (стримує модуль вагів)
    save_total_limit=3,    # will limit the total amount of checkpoints. Deletes the older checkpoints in output_dir.
    num_train_epochs=4,    # Total number of training epochs to perform
    predict_with_generate=True,   # Whether to use generate to calculate generative metrics
                                  # https://discuss.huggingface.co/t/what-does-generate-with-predict-true-actually-do/8685
                                  # basically just saying that we are using a custom eval function and not the built-in loss
    #fp16=True,  # Whether to use fp16 16-bit (mixed) precision training instead of 32-bit training
                # Only for GPU, apparently. Needs CUDA.
    #bf16=True,
    half_precision_backend="cpu_amp"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=test_tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


---

In [34]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: article, highlights. If article, highlights are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 28712
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 7180
  Number of trainable parameters = 60506624


  0%|          | 0/7180 [00:00<?, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


RuntimeError: [enforce fail at C:\cb\pytorch_1000000000000\work\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 1844430848 bytes.

In [ ]:
bow1

CountVectorizer(max_features=2000, min_df=2, stop_words='english')